In [2]:
!pip install beautifulsoup4

In [3]:
!pip install lxml
!pip install html5lib
!pip install requests

import the required packages

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd # library for data analsysis

setup the url of the website

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

creates a soup object of the type beautifulsoup and reteieves the wikipedia page to the object

In [4]:
soup = BeautifulSoup(source, 'lxml')
tables = soup.find("table")

extract the content from website and store it in pandas dataframe

In [5]:
data = []
columns = []

for index, tr in enumerate(tables.find_all('tr')):
    header = []
    for td in tr.find_all(['th','td']):
        header.append(td.text.strip())
    
    if (index == 0):
        columns = header
    else:
        data.append(header)

df = pd.DataFrame(data = data,columns = columns)

remove any row where there is no value for Borough

In [6]:
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)

Groupby the neighbourhoods that has the same post code and borough

In [7]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

Any neighbourhood with not assigned value is replaces with it's respective borough name

In [8]:
df['Neighbourhood'].replace("Not assigned", df["Borough"],inplace=True)

get the row count of dataframe

In [17]:
df.shape

(103, 3)

read csv file to lat and longtitudes

In [21]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


set lat and longtitudes based on postcode

In [27]:
coordinates.rename(columns={"Postal Code": "Postcode"}, inplace=True)
coordinates

df_new = df.merge(coordinates, on="Postcode",  how="left")
df_new.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


get coordinates of toronto

In [74]:
import folium
from geopy.geocoders import Nominatim
address = 'Toronto'

geolocator = Nominatim(user_agent="geo_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinate of Toronto is  {}, {}.'.format(latitude, longitude))

The coordinate of Toronto is  43.653963, -79.387207.


create map of toronto

In [75]:

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

#add markers 
for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5).add_to(map_toronto)  
    
map_toronto

In [76]:
# filter borough names that contain the word Toronto
borough_names = list(df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
print(borough_with_toronto)

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']


create a new df for above boroughs

In [77]:
df_new_toronto = df_new[df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
df_new_toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


recrete the map for new dataframe

In [78]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_new_toronto['Latitude'], df_new_toronto['Longitude'], df_new_toronto['Borough'], df_new_toronto['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5).add_to(map_toronto)  
    
map_toronto

In [79]:
CLIENT_ID = '3VZLKQBERMW0RIMFMC2JJWLRJYDL4F55VM1EHX21GRMREIVE' # your Foursquare ID
CLIENT_SECRET = 'RTYKXRTGF5XCNV4HIZOIT2AQ54L4IAQ111WWQJA3ZL2GUF2K' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighbourhood in zip(df_new_toronto['Latitude'], df_new_toronto['Longitude'], df_new_toronto['Postcode'], df_new_toronto['Borough'], df_new_toronto['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighbourhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

Your credentails:
CLIENT_ID: 3VZLKQBERMW0RIMFMC2JJWLRJYDL4F55VM1EHX21GRMREIVE
CLIENT_SECRET:RTYKXRTGF5XCNV4HIZOIT2AQ54L4IAQ111WWQJA3ZL2GUF2K


In [80]:
df_newVenues = pd.DataFrame(venues)
# define the column names
df_newVenues.columns = ['Postcode', 'Borough', 'Neighbourhoods', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(df_newVenues.shape)
df_newVenues.head()

(1692, 9)


,Postcode,Borough,Neighbourhoods,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


perfrom one hot encoding

In [81]:
df_newVenues.groupby(["Postcode", "Borough", "Neighbourhoods"]).count()
print('There are {} uniques categories.'.format(len(df_newVenues['VenueCategory'].unique())))
df_newVenues['VenueCategory'].unique()[:50]

# one hot encoding
df_toronto_onehot = pd.get_dummies(df_newVenues[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
df_toronto_onehot['Postcode'] = df_newVenues['Postcode'] 
df_toronto_onehot['Borough'] = df_newVenues['Borough'] 
df_toronto_onehot['Neighbourhoods'] = df_newVenues['Neighbourhoods'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(df_toronto_onehot.columns[-3:]) + list(df_toronto_onehot.columns[:-3])
df_toronto_onehot = df_toronto_onehot[fixed_columns]

print(df_toronto_onehot.shape)
df_toronto_onehot.head()

df_toronto_grouped = df_toronto_onehot.groupby(["Postcode", "Borough", "Neighbourhoods"]).mean().reset_index()

print(df_toronto_grouped.shape)
df_toronto_grouped.head()

There are 234 uniques categories.
(1692, 237)
(38, 237)


,Postcode,Borough,Neighbourhoods,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,M4K,East Toronto,"The Danforth West, Riverdale",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.023256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023256
2,M4L,East Toronto,"The Beaches West, India Bazaar",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,M4M,East Toronto,Studio District,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025000
4,M4N,Central Toronto,Lawrence Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [83]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['Postcode', 'Borough', 'Neighbourhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = df_toronto_grouped['Postcode']
neighborhoods_venues_sorted['Borough'] = df_toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighbourhoods'] = df_toronto_grouped['Neighbourhoods']

for ind in np.arange(df_toronto_grouped.shape[0]):
    row_categories = df_toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(38, 13)


,Postcode,Borough,Neighbourhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Health Food Store,Coffee Shop,Pub,Neighborhood,Cosmetics Shop,Convenience Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Grocery Store,Brewery,Bubble Tea Shop,Restaurant,Caribbean Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",Park,Pet Store,Sushi Restaurant,Brewery,Burger Joint,Burrito Place,Sandwich Place,Pub,Coffee Shop,Movie Theater
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant,Gastropub,Chinese Restaurant,Fish Market,Juice Bar,Bookstore
4,M4N,Central Toronto,Lawrence Park,Park,Swim School,Bus Line,Dim Sum Restaurant,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


Run K Means

In [84]:
from sklearn.cluster import KMeans

#number of clusters
kclusters = 5

toronto_clustering = df_toronto_grouped.drop(["Postcode", "Borough", "Neighbourhoods"], 1)

# k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

kmeans.labels_[0:10]
toronto_merged = df_new_toronto.copy()
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighbourhoods"], 1).set_index("Postcode"), on="Postcode")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged.head()

(38, 16)
(38, 16)


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Coffee Shop,Pub,Neighborhood,Cosmetics Shop,Convenience Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
20,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,0,Coffee Shop,Hotel,Café,American Restaurant,Restaurant,Gastropub,Deli / Bodega,Burger Joint,Steakhouse,Gym
21,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Deli / Bodega,Italian Restaurant,Seafood Restaurant,Steakhouse,Bakery
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Café,Sandwich Place,Coffee Shop,Pizza Place,BBQ Joint,American Restaurant,Pub,Indian Restaurant,Jewish Restaurant,Cheese Shop
25,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049,0,Café,Coffee Shop,Bar,Japanese Restaurant,Restaurant,Bookstore,Bakery,Dessert Shop,Chinese Restaurant,Pub


Visualize the clustering

In [86]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#add markers
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postcode'], toronto_merged['Borough'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=8,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.5).add_to(map_clusters)
       
map_clusters

In [87]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]] #cluster1

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Health Food Store,Coffee Shop,Pub,Neighborhood,Cosmetics Shop,Convenience Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
20,Downtown Toronto,0,Coffee Shop,Hotel,Café,American Restaurant,Restaurant,Gastropub,Deli / Bodega,Burger Joint,Steakhouse,Gym
21,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Deli / Bodega,Italian Restaurant,Seafood Restaurant,Steakhouse,Bakery
24,Central Toronto,0,Café,Sandwich Place,Coffee Shop,Pizza Place,BBQ Joint,American Restaurant,Pub,Indian Restaurant,Jewish Restaurant,Cheese Shop
25,Downtown Toronto,0,Café,Coffee Shop,Bar,Japanese Restaurant,Restaurant,Bookstore,Bakery,Dessert Shop,Chinese Restaurant,Pub
26,Downtown Toronto,0,Bar,Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bakery,Coffee Shop,Mexican Restaurant,Vietnamese Restaurant,Caribbean Restaurant,Arts & Crafts Store
27,Downtown Toronto,0,Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Plane
19,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Pizza Place,Brewery,Restaurant,Scenic Lookout,Bakery
28,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Hotel,Seafood Restaurant,Fast Food Restaurant,Beer Bar,Italian Restaurant,Cocktail Bar,Art Gallery
30,Downtown Toronto,0,Grocery Store,Café,Park,Nightclub,Baby Store,Italian Restaurant,Restaurant,Diner,Coffee Shop,Convenience Store
